# 4. Feature Selection

In [1]:
# libs:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',60)

#Testes estatísticos
import scipy
from scipy.stats import stats
import statsmodels.api as sm

from feature_engine.categorical_encoders import CountFrequencyCategoricalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

#fearture selection:
from sklearn.feature_selection import chi2
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import chi2_contingency
from boruta import BorutaPy
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings("ignore")

Vamos utilizar o método de votação marjoritáia. 3 Propostas de seleção.
<br>
* Teste estatístico
* RFECV
* Boruta<br>

As features que forem mantidas por pelo menos 2 algoritmos serão utilizadas no processo de modelagem

## Dados Constantes

In [2]:
DATA_INPUT_ENG = '../data/DATA_train_eng.csv'
DATA_OUTPUT_FEAT_SELECTED = '../data/DATA_feature_selected.csv'

In [3]:
df_train = pd.read_csv(DATA_INPUT_ENG)

**Dropar variáveis que não vamos utilizar**

In [4]:
# Vamos dropar algumas colunas
df_train = df_train.drop({'Unnamed: 0','ID', 'Check-In', 'Check-Out', 'Preço com taxas', 'Diária com taxas'}, axis=1)

In [5]:
df_train.head()

,Localização,Número Hóspedes,Número Quartos,Número Camas,Número Banheiros,Avaliação,Número Comentários,Preço/Noite,Piscina,Lava-louças,Wi-Fi,Café da Manhã,Máquina de Lavar,Estacionamento gratuito,Academia,Jacuzzi,Secadora,Entrada/saída para esquis,Cozinha,Permitido animais,Lareira interna,Ar-condicionado,Self check-In,Elevador,Novo preço mais baixo,Raridade,Banheiro Compartilhado,Taxa,Vista,Demanda,É_feriado,É outilier,Mes,Dia,Semana_do_ano,Semana de Feriado
0,0.029461,4,1,3,1,5.00,18,184.0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,False,60.0,0,0,0,0,3,5,9,0.0
1,0.042766,4,1,4,1,4.83,41,198.0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,False,30.0,0,0,0,0,3,19,11,0.0
2,0.003089,10,3,8,2,4.43,7,351.0,0,0,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,False,0.0,0,0,0,0,3,5,9,0.0
3,0.018769,4,1,2,1,4.40,5,191.0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,False,60.0,0,0,0,0,3,26,12,0.0
4,0.018769,4,1,4,1,5.00,3,124.0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,False,58.0,0,0,0,0,2,26,8,0.0


In [6]:
df_train.shape

(4209, 36)

In [7]:
# Divisão para os testes Wrap methods
X_train = df_train.drop('Preço/Noite', axis=1)
y_train = df_train['Preço/Noite']

**Tratamento de alguns tipos:**

In [8]:
df_train.dtypes

Localização                  float64
Número Hóspedes                int64
Número Quartos                 int64
Número Camas                   int64
Número Banheiros               int64
Avaliação                    float64
Número Comentários             int64
Preço/Noite                  float64
Piscina                        int64
Lava-louças                    int64
Wi-Fi                          int64
Café da Manhã                  int64
Máquina de Lavar               int64
Estacionamento gratuito        int64
Academia                       int64
Jacuzzi                        int64
Secadora                       int64
Entrada/saída para esquis      int64
Cozinha                        int64
Permitido animais              int64
Lareira interna                int64
Ar-condicionado                int64
Self check-In                  int64
Elevador                       int64
Novo preço mais baixo          int64
Raridade                       int64
Banheiro Compartilhado          bool
T

**Divisão entre as variáveis númericas e categóricas**

In [9]:
features_num = ['Número Hóspedes', 'Número Quartos', 'Número Camas','Número Banheiros', 
                 'Avaliação', 'Número Comentários', 'Taxa', 'Dia', 'Mes']

In [10]:
features_cat = [ 'Localização', 'Academia','Ar-condicionado', 'Cozinha', 'Elevador',
                       'Estacionamento gratuito', 'Máquina de Lavar', 'Permitido animais',
                       'Piscina', 'Secadora', 'Self check-In', 'Wi-Fi', 'Café da Manhã',
                       'Entrada/saída para esquis', 'Jacuzzi', 'Lareira interna',
                       'Lava-louças', 'Novo preço mais baixo', 'Raridade',
                       'Banheiro Compartilhado','Vista', 'Demanda', 'É_feriado',
                       'É outilier', 'Semana_do_ano', 'Semana de Feriado']

In [11]:
Target = ['Preço/Noite']

In [12]:
# Conversão de Tipos
for col in features_cat:
    df_train[col] = df_train[col].astype(str)

In [13]:
df_train.dtypes

Localização                   object
Número Hóspedes                int64
Número Quartos                 int64
Número Camas                   int64
Número Banheiros               int64
Avaliação                    float64
Número Comentários             int64
Preço/Noite                  float64
Piscina                       object
Lava-louças                   object
Wi-Fi                         object
Café da Manhã                 object
Máquina de Lavar              object
Estacionamento gratuito       object
Academia                      object
Jacuzzi                       object
Secadora                      object
Entrada/saída para esquis     object
Cozinha                       object
Permitido animais             object
Lareira interna               object
Ar-condicionado               object
Self check-In                 object
Elevador                      object
Novo preço mais baixo         object
Raridade                      object
Banheiro Compartilhado        object
T

Tudo Pronto!

## 4.1 Testes Estatísticos:

### Features Númericas:

### 1. Teste de Correlação

In [14]:
# Teste de correlação
def teste_correlacao(df, Target, features, alpha=0.05):
    """
    df = Dataframe
    Target = target
    features = numeric features to be tested
    Return list of variable which are correlationed with target
    """
    correlation = []
    no_correlation = []
    for col in features:
        cor, p = stats.spearmanr(df_train[col],df_train[Target])
        if p <= alpha:
            correlation.append(col)
        else:
            no_correlation.append(col)
            
    return correlation

In [15]:
correlation = teste_correlacao(df_train, Target, features_num)
correlation

['Número Hóspedes',
 'Número Quartos',
 'Número Camas',
 'Número Banheiros',
 'Avaliação',
 'Número Comentários',
 'Taxa',
 'Dia',
 'Mes']

### Features categóricas:

### Chi2

In [16]:
def teste_chi2_(df, Target='Target', features_cat=features_cat, aplha=0.05):
    """
    Chi2 test
    df = DataFrame
    Target = Target string
    features = List of categorical features
    aplha = indice of significance
    Return a list of variable which has passed for Chi2 test
    """
    from scipy.stats import chi2_contingency
    p_values_cat_features = {}
    for col in features_cat:
        # Cria tabela de contingencia
        df_cross = pd.crosstab(df[col], df[Target])
        # Aplica o teste e extrai o p-valor
        p_value = scipy.stats.chi2_contingency(df_cross)[1]
        # Armazena coluna e p-valor em um dict
        p_values_cat_features[col] = p_value
        
    p_values_cat_features = pd.Series(p_values_cat_features)
    filter_cat_features = p_values_cat_features[p_values_cat_features < aplha].index.tolist()
        
    return filter_cat_features

In [17]:
chi2 = teste_chi2_(df_train, Target = 'Preço/Noite', features_cat=features_cat)

In [18]:
chi2

['Localização',
 'Academia',
 'Ar-condicionado',
 'Cozinha',
 'Elevador',
 'Estacionamento gratuito',
 'Máquina de Lavar',
 'Permitido animais',
 'Piscina',
 'Secadora',
 'Self check-In',
 'Wi-Fi',
 'Café da Manhã',
 'Entrada/saída para esquis',
 'Jacuzzi',
 'Lareira interna',
 'Lava-louças',
 'Banheiro Compartilhado',
 'Vista',
 'Demanda',
 'É_feriado',
 'Semana_do_ano',
 'Semana de Feriado']

Pelo processo de correlação, todas passaram pelo teste, porém pelo teste de Multicolinearidade. Número de quartos e número de hospedes não passaram.

## 4.2 RFECV

Para este teste vamos utilizar o Dataframe encode

In [19]:
def teste_rfe(X_train, y_train):
    """
    RFE Test
    X_train = Dataframe with all features to be tested and not target
    y_train = Target
    Return a list of varibales which has passed for the test
    """
    from sklearn.feature_selection import RFECV
    from sklearn.ensemble import RandomForestRegressor
    rfecv_RFC = RFECV(estimator=RandomForestRegressor(n_jobs = -1, max_depth = 5), scoring='neg_mean_squared_error')
    rfecv_RFC.fit(X_train,y_train)
    mask_RFC = rfecv_RFC.support_
    cols_selected_RFE = X_train.columns.tolist()
    
    return cols_selected_RFE

In [20]:
rfe = teste_rfe(X_train, y_train)

In [21]:
rfe

['Localização',
 'Número Hóspedes',
 'Número Quartos',
 'Número Camas',
 'Número Banheiros',
 'Avaliação',
 'Número Comentários',
 'Piscina',
 'Lava-louças',
 'Wi-Fi',
 'Café da Manhã',
 'Máquina de Lavar',
 'Estacionamento gratuito',
 'Academia',
 'Jacuzzi',
 'Secadora',
 'Entrada/saída para esquis',
 'Cozinha',
 'Permitido animais',
 'Lareira interna',
 'Ar-condicionado',
 'Self check-In',
 'Elevador',
 'Novo preço mais baixo',
 'Raridade',
 'Banheiro Compartilhado',
 'Taxa',
 'Vista',
 'Demanda',
 'É_feriado',
 'É outilier',
 'Mes',
 'Dia',
 'Semana_do_ano',
 'Semana de Feriado']

## 4.3 Boruta

In [22]:
def teste_boruta(X_train, y_train):
    """
    Boruta Test - Return features which was aproved
    X_train = Dataframe with all features to be tested and not target
    y_train = Target
    Return a list of varibales which has passed for the test
    """
    from boruta import BorutaPy
    from sklearn.ensemble import RandomForestRegressor
    # -1 indica para o python usar todo o processador 
    boruta_selector = BorutaPy(RandomForestRegressor(n_jobs = -1, max_depth = 5), n_estimators = 50, max_iter=100, random_state = 0)
    boruta_selector.fit(np.array(X_train), np.array(y_train))
    boruta_selector.support_
    boruta_selector.support_weak_
    features_boruta = X_train.loc[:, boruta_selector.support_].columns.tolist()

    return features_boruta 

In [23]:
boruta = teste_boruta(X_train, y_train)

In [24]:
boruta

['Localização',
 'Número Hóspedes',
 'Número Quartos',
 'Número Camas',
 'Número Banheiros',
 'Avaliação',
 'Número Comentários',
 'Piscina',
 'Academia',
 'Jacuzzi',
 'Secadora',
 'Permitido animais',
 'Taxa',
 'Mes',
 'Semana_do_ano']

# 4.4 Variáveis Selecionadas

### Testes estatísticos:

In [25]:
correlation

['Número Hóspedes',
 'Número Quartos',
 'Número Camas',
 'Número Banheiros',
 'Avaliação',
 'Número Comentários',
 'Taxa',
 'Dia',
 'Mes']

In [26]:
chi2

['Localização',
 'Academia',
 'Ar-condicionado',
 'Cozinha',
 'Elevador',
 'Estacionamento gratuito',
 'Máquina de Lavar',
 'Permitido animais',
 'Piscina',
 'Secadora',
 'Self check-In',
 'Wi-Fi',
 'Café da Manhã',
 'Entrada/saída para esquis',
 'Jacuzzi',
 'Lareira interna',
 'Lava-louças',
 'Banheiro Compartilhado',
 'Vista',
 'Demanda',
 'É_feriado',
 'Semana_do_ano',
 'Semana de Feriado']

In [70]:
#vif

In [71]:
#statistic_test_num = list(set(correlation).intersection(set(vif)))

In [72]:
def feature_results___(correlation, chi2, rfe, boruta):
    
    # Junção dos testes de correlação e Chi2
    statistic_test = correlation + chi2
    # Interserção entre testes estatísticos e RFE
    statistic_test_rfe = list(set(statistic_test).intersection(set(rfe)))
    # Quem passou no primeiro que também passou no boruta
    feature_selection = list(set(statistic_test_rfe).intersection(set(boruta)))
    
    return feature_selection

In [30]:
def feature_results(correlation, chi2, rfe, boruta):
    """
    Return a list which has passed for three testes
    correlation = set of result from corrrelation teste
    chi2 = set of features which has passed from chi2 teste
    rfe = set of features which has passed from rfe teste
    boruta = set of features which has passed from boruta teste
    """
    # Junção dos testes de correlação e Chi2
    statistic_test = correlation + chi2
    # Interserção entre testes estatísticos e RFE
    statistic_test_rfe = list(set(statistic_test).intersection(set(rfe)))
    # Interserção entre testes estatísticos e boruta
    statistic_test_boruta = list(set(statistic_test).intersection(set(boruta)))
    # Interserção entre rfe e boruta
    rfe_boruta = list(set(rfe).intersection(set(boruta)))
    
    # Quem passou no primeiro que também passou no boruta
    feature_selection = set(statistic_test_rfe) & set(statistic_test_boruta) & set(rfe_boruta)
    
    return feature_selection

In [31]:
feature_results(correlation, chi2, rfe, boruta)

{'Academia',
 'Avaliação',
 'Jacuzzi',
 'Localização',
 'Mes',
 'Número Banheiros',
 'Número Camas',
 'Número Comentários',
 'Número Hóspedes',
 'Número Quartos',
 'Permitido animais',
 'Piscina',
 'Secadora',
 'Semana_do_ano',
 'Taxa'}

In [32]:
feature_selection = feature_results(correlation, chi2, rfe, boruta)

In [33]:
feature_selection

{'Academia',
 'Avaliação',
 'Jacuzzi',
 'Localização',
 'Mes',
 'Número Banheiros',
 'Número Camas',
 'Número Comentários',
 'Número Hóspedes',
 'Número Quartos',
 'Permitido animais',
 'Piscina',
 'Secadora',
 'Semana_do_ano',
 'Taxa'}

In [34]:
feature_selection_num = []
feature_selection_cat = []
count = 0
for i in feature_selection:
    if i in features_num:
        feature_selection_num.append(i)
    if i in features_cat:
        feature_selection_cat.append(i)
        count += 1

In [35]:
feature_selection_num

['Número Comentários',
 'Número Camas',
 'Número Quartos',
 'Número Banheiros',
 'Taxa',
 'Mes',
 'Número Hóspedes',
 'Avaliação']

In [36]:
feature_selection_cat

['Semana_do_ano',
 'Jacuzzi',
 'Academia',
 'Secadora',
 'Localização',
 'Piscina',
 'Permitido animais']

# 4.5 Teste de Multicolineariade

In [37]:
def multicolinearidade(df, features):
    """
    V-Cramer test
    X = DataFrame without target
    features = numerical features to be tested
    Return a Dataframe with VIF score
    """  
    # VIF dataframe 
    vif_data = pd.DataFrame() 
    vif_data["feature"] = df.loc[:,features].columns 
    # calculando VIF 
    vif_data["VIF"] = [variance_inflation_factor(df.loc[:,features].values, i) for i in range(len(df.loc[:,features].columns))] 
    
    return vif_data.sort_values(by='VIF', ascending=False)

In [38]:
multicolinearidade(X_train, feature_selection_num)

,feature,VIF
7,Avaliação,24.386740
2,Número Quartos,18.434627
5,Mes,15.014006
3,Número Banheiros,14.024909
6,Número Hóspedes,12.705066
1,Número Camas,6.224634
4,Taxa,4.763461
0,Número Comentários,2.031651


In [41]:
feature_selection_num.remove('Avaliação')

In [42]:
multicolinearidade(X_train, feature_selection_num)

,feature,VIF
2,Número Quartos,18.431163
3,Número Banheiros,13.868337
6,Número Hóspedes,12.101648
1,Número Camas,6.211218
5,Mes,5.797885
4,Taxa,4.320247
0,Número Comentários,1.935669


In [43]:
feature_selection_num.remove('Número Quartos')

In [44]:
multicolinearidade(X_train, feature_selection_num)

,feature,VIF
5,Número Hóspedes,10.565242
2,Número Banheiros,8.419417
1,Número Camas,5.918743
4,Mes,5.793528
3,Taxa,4.267290
0,Número Comentários,1.933112


In [45]:
vif = multicolinearidade(X_train, feature_selection_num)

In [46]:
def vif_list(vif):
    """
    Tranforming resut VIF test into a list of variables
    vif = DataFrame from teste_vif
    Return a list of variables 
    """
    vif = vif.pivot_table(columns='feature')
    vif = vif.columns.tolist()
    
    return vif

In [47]:
vif = vif_list(vif)

In [48]:
# Variáveis sem multicolinearidade
vif

['Mes',
 'Número Banheiros',
 'Número Camas',
 'Número Comentários',
 'Número Hóspedes',
 'Taxa']

In [ ]:
# Agora vamos juntar estas variáveis áquela que dropamos ante de passar no VIF.

Finalizado todos os testes, Temos um conjunto sem multicolinearidade para o modelo.

In [49]:
df_train = df_train[vif + feature_selection_cat + Target]

In [50]:
df_train

,Mes,Número Banheiros,Número Camas,Número Comentários,Número Hóspedes,Taxa,Semana_do_ano,Jacuzzi,Academia,Secadora,Localização,Piscina,Permitido animais,Preço/Noite
0,3,1,3,18,4,60.0,9,0,0,0,0.02946067949631741,1,0,184.0
1,3,1,4,41,4,30.0,11,0,0,0,0.04276550249465432,1,0,198.0
2,3,2,8,7,10,0.0,9,0,0,0,0.0030886196246139226,0,0,351.0
3,3,1,2,5,4,60.0,12,0,0,0,0.01876930387265384,0,1,191.0
4,2,1,4,3,4,58.0,8,0,0,0,0.01876930387265384,0,1,124.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,3,3,4,34,10,75.0,11,0,0,0,0.005939653124257543,1,0,581.0
4205,3,1,3,16,4,49.5,9,0,0,0,0.0033262057495842237,0,1,247.0
4206,3,1,2,6,6,10.0,9,0,0,0,0.011879306248515086,0,1,150.0
4207,3,1,6,38,6,10.0,11,0,0,0,0.009978617248752672,0,0,103.0


In [51]:
print(len(df_train.columns))

14


In [ ]:
# criar função de junção das features selecionadas no dataframe

# Export

In [52]:
DATA_OUTPUT_FEAT_SELECTED_V2 = '../data/DATA_feature_selected_V2.csv'

In [53]:
df_train.to_csv(DATA_OUTPUT_FEAT_SELECTED_V2)